In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

image_dir = "UTKFace/UTKFace"  # This is the correct path

data = []
age_labels = []
gender_labels = []

for filename in os.listdir(image_dir):
    try:
        if not filename.endswith(".jpg"):
            continue

        age, gender, *_ = filename.split("_")
        img_path = os.path.join(image_dir, filename)
        img = cv2.imread(img_path)
        if img is None:
            continue
        img = cv2.resize(img, (64, 64))

        data.append(img)
        age_labels.append(int(age))
        gender_labels.append(int(gender))
    except Exception as e:
        print(f"Skipping {filename}: {e}")
        continue

print(f"✅ Total valid images loaded: {len(data)}")

X = np.array(data) / 255.0
y_age = np.array(age_labels)
y_gender = np.array(gender_labels)

# Split into train/test
X_train, X_test, y_age_train, y_age_test, y_gender_train, y_gender_test = train_test_split(
    X, y_age, y_gender, test_size=0.2, random_state=42
)

print("✅ Data preprocessed and ready for model training.")


✅ Total valid images loaded: 23708
✅ Data preprocessed and ready for model training.


In [4]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

# Input Layer
input_layer = Input(shape=(64, 64, 3))

# Shared Convolutional Base
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_layer)
x = MaxPooling2D((2, 2))(x)
x = BatchNormalization()(x)

x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2))(x)
x = BatchNormalization()(x)

x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2))(x)
x = BatchNormalization()(x)

x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2))(x)
x = BatchNormalization()(x)

x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)

# Output 1: Gender Classification
gender_output = Dense(1, activation='sigmoid', name='gender_output')(x)

# Output 2: Age Regression
age_output = Dense(1, activation='linear', name='age_output')(x)

# Define Model
model = Model(inputs=input_layer, outputs=[gender_output, age_output])

# Compile Model
model.compile(
    optimizer='adam',
    loss={
        'gender_output': 'binary_crossentropy',
        'age_output': 'mae'
    },
    metrics={
        'gender_output': 'accuracy',
        'age_output': 'mae'
    }
)

model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 64, 64, 3)         │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d (Conv2D)               │ (None, 64, 64, 32)        │             896 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d (MaxPooling2D)  │ (None, 32, 32, 32)        │               0 │ conv2d[0][0]               │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization           │ (None, 32, 32, 32)        │             128 │ max_pooling2d[0][0]        │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_1 (Conv2D)             │ (None, 32, 32, 64)        │          18,496 │ batch_normalization[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_1               │ (None, 16, 16, 64)        │               0 │ conv2d_1[0][0]             │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_1         │ (None, 16, 16, 64)        │             256 │ max_pooling2d_1[0][0]      │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_2 (Conv2D)             │ (None, 16, 16, 128)       │          73,856 │ batch_normalization_1[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_2               │ (None, 8, 8, 128)         │               0 │ conv2d_2[0][0]             │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_2         │ (None, 8, 8, 128)         │             512 │ max_pooling2d_2[0][0]      │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_3 (Conv2D)             │ (None, 8, 8, 256)         │         295,168 │ batch_normalization_2[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_3               │ (None, 4, 4, 256)         │               0 │ conv2d_3[0][0]             │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_3         │ (None, 4, 4, 256)         │           1,024 │ max_pooling2d_3[0][0]      │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ flatten (Flatten)             │ (None, 4096)              │               

 Total params: 915,010 (3.49 MB)

 Trainable params: 914,050 (3.49 MB)

 Non-trainable params: 960 (3.75 KB)

In [3]:
# history = model.fit(
#     X_train,
#     {'gender_output': y_gender_train, 'age_output': y_age_train},
#     validation_split=0.1,
#     epochs=20,
#     batch_size=64
# )


Epoch 1/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 46s 156ms/step - age_output_loss: 13.5902 - age_output_mae: 13.5902 - gender_output_accuracy: 0.6966 - gender_output_loss: 1.0561 - loss: 14.6464 - val_age_output_loss: 13.6273 - val_age_output_mae: 13.6271 - val_gender_output_accuracy: 0.7132 - val_gender_output_loss: 0.5378 - val_loss: 14.1650
Epoch 2/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 41s 152ms/step - age_output_loss: 9.3962 - age_output_mae: 9.3962 - gender_output_accuracy: 0.7986 - gender_output_loss: 0.5221 - loss: 9.9183 - val_age_output_loss: 12.2695 - val_age_output_mae: 12.2659 - val_gender_output_accuracy: 0.8392 - val_gender_output_loss: 0.3547 - val_loss: 12.6216
Epoch 3/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 41s 155ms/step - age_output_loss: 8.0955 - age_output_mae: 8.0955 - gender_output_accuracy: 0.8249 - gender_output_loss: 0.4029 - loss: 8.4984 - val_age_output_loss: 7.1356 - val_age_output_mae: 7.1265 - val_gender_output_accuracy: 0.8635 - val_gender_output_loss: 0.3254 - val_loss: 7.452

In [5]:
import matplotlib.pyplot as plt

# Gender accuracy
plt.plot(history.history['gender_output_accuracy'], label='Gender Accuracy')
plt.plot(history.history['val_gender_output_accuracy'], label='Val Gender Accuracy')
plt.legend()
plt.title("Gender Accuracy")
plt.show()

# Age MAE
plt.plot(history.history['age_output_mae'], label='Age MAE')
plt.plot(history.history['val_age_output_mae'], label='Val Age MAE')
plt.legend()
plt.title("Age MAE")
plt.show()


NameError: name 'history' is not defined

In [4]:
results = model.evaluate(X_test, {'gender_output': y_gender_test, 'age_output': y_age_test})
print("\nTest Loss & Metrics:")
for name, value in zip(model.metrics_names, results):
    print(f"{name}: {value:.4f}")


149/149 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - age_output_loss: 6.2934 - age_output_mae: 6.2935 - gender_output_accuracy: 0.8908 - gender_output_loss: 0.2736 - loss: 6.5672

Test Loss & Metrics:
loss: 6.5641
compile_metrics: 0.2559
gender_output_loss: 6.2990
age_output_loss: 6.3071


In [5]:
import numpy as np

# Predict gender and age on test data
gender_pred_prob, age_pred = model.predict(X_test)

# Since gender is sigmoid output, convert to binary labels
gender_pred = (gender_pred_prob > 0.5).astype(int).reshape(-1)


149/149 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step


In [6]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

print("Gender Classification Metrics:")

cm = confusion_matrix(y_gender_test, gender_pred)
print("Confusion Matrix:\n", cm)

acc = accuracy_score(y_gender_test, gender_pred)
prec = precision_score(y_gender_test, gender_pred)
rec = recall_score(y_gender_test, gender_pred)
f1 = f1_score(y_gender_test, gender_pred)

print(f"Accuracy: {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall: {rec:.4f}")
print(f"F1 Score: {f1:.4f}")


Gender Classification Metrics:
Confusion Matrix:
 [[2270  215]
 [ 293 1964]]
Accuracy: 0.8929
Precision: 0.9013
Recall: 0.8702
F1 Score: 0.8855


In [7]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

print("\nAge Regression Metrics:")

r2 = r2_score(y_age_test, age_pred)
mae = mean_absolute_error(y_age_test, age_pred)
mse = mean_squared_error(y_age_test, age_pred)

print(f"R² Score: {r2:.4f}")
print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")



Age Regression Metrics:
R² Score: 0.8021
MAE: 6.3071
MSE: 78.5671


In [8]:
model.save('age_gender_model3.keras', include_optimizer=False)


In [9]:
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanAbsoluteError

model = load_model('age_gender_model1.keras', compile=False)

model.compile(
    optimizer='adam',
    loss={
        'gender_output': 'binary_crossentropy',
        'age_output': MeanAbsoluteError()
    },
    metrics={
        'gender_output': 'accuracy',
        'age_output': MeanAbsoluteError()
    }
)


In [2]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanAbsoluteError

# Load the trained model and compile explicitly
model = load_model('age_gender_model3.keras', compile=False)
model.compile(
    optimizer='adam',
    loss={
        'gender_output': 'binary_crossentropy',
        'age_output': MeanAbsoluteError()
    },
    metrics={
        'gender_output': 'accuracy',
        'age_output': MeanAbsoluteError()
    }
)

# Load Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def preprocess_face(face_img):
    """Resize, normalize, and reshape face image for prediction."""
    face_img = cv2.resize(face_img, (64, 64))
    face_img = face_img.astype('float32') / 255.0
    face_img = np.expand_dims(face_img, axis=0)  # Shape: (1, 64, 64, 3)
    return face_img

# Start webcam capture
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        # Extract the face ROI from original frame
        face_img = frame[y:y+h, x:x+w]

        # Preprocess for model
        input_img = preprocess_face(face_img)

        # Predict gender and age
        gender_pred_prob, age_pred = model.predict(input_img)
        gender_label = "Female" if gender_pred_prob[0][0] > 0.5 else "Male"
        age_label = int(age_pred[0][0])

        # Draw bounding box on the face
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        # Text position above the box
        text = f"{gender_label}, Age: {age_label}"
        (text_width, text_height), baseline = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
        cv2.rectangle(frame, (x, y - text_height - 10), (x + text_width, y), (0, 255, 0), -1)  # Filled box for text
        cv2.putText(frame, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)

    # Display the frame
    cv2.imshow('Age & Gender Prediction', frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 669ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step


In [3]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanAbsoluteError
from collections import deque

# Load the trained model
model = load_model('age_gender_model3.keras', compile=False)
model.compile(
    optimizer='adam',
    loss={
        'gender_output': 'binary_crossentropy',
        'age_output': MeanAbsoluteError()
    },
    metrics={
        'gender_output': 'accuracy',
        'age_output': MeanAbsoluteError()
    }
)

# Load Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# For stabilizing predictions
age_history = {}
gender_history = {}
MAX_HISTORY = 10

def preprocess_face(face_img):
    face_img = cv2.resize(face_img, (64, 64))
    face_img = face_img.astype('float32') / 255.0
    return np.expand_dims(face_img, axis=0)

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for idx, (x, y, w, h) in enumerate(faces):
        face_img = frame[y:y+h, x:x+w]
        input_img = preprocess_face(face_img)

        gender_pred_prob, age_pred = model.predict(input_img, verbose=0)
        gender_label = "Female" if gender_pred_prob[0][0] > 0.5 else "Male"
        age_value = int(age_pred[0][0])

        # Keep prediction history
        if idx not in age_history:
            age_history[idx] = deque(maxlen=MAX_HISTORY)
            gender_history[idx] = deque(maxlen=MAX_HISTORY)

        age_history[idx].append(age_value)
        gender_history[idx].append(gender_label)

        # Stable prediction
        stable_age = int(np.median(age_history[idx]))
        stable_gender = max(set(gender_history[idx]), key=gender_history[idx].count)

        text = f"{stable_gender}, Age: {stable_age}"
        (tw, th), _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
        cv2.rectangle(frame, (x, y - th - 10), (x + tw, y), (0, 255, 0), -1)
        cv2.putText(frame, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv2.imshow("Age & Gender Prediction (Stable)", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
